# Estimator 
### Last Modified: September 21, 2020

To run, press SHIFT+ENTER on each cell. <br>
To get to next cell, press the down arrow after clicking on a blank spot of the cell. <br>
<b> Likewise, you can click on "Restart & Run All" under the Kernel menu (may need to re-run cells with dropdown boxes)</b><br>
Cells can be toggled on and off by running the cell preceding this message.
## If asked to "Try and re-run cell", then click on a blank spot of the cell until no dropdown box or button is selected and press SHIFT+ENTER

In [ ]:
# Widgets
from IPython.display import display, clear_output, HTML, FileLink
import ipywidgets as widgets
from ipywidgets import Layout

# Imports
import pandas as pd
import numpy as np
import math

import io
import requests

import time
from datetime import date

import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Variables
scope = "https://spreadsheets.google.com/feeds"
credentials = ServiceAccountCredentials.from_json_keyfile_name("matcher-272116-8801bce55bcb.json", scope)
gs = gspread.authorize(credentials)

estimator_sheet_url = "https://docs.google.com/spreadsheets/d/1EhbS_vM28BQWW2SVCS1RefWD9tzDrWbYT2MdSDlobVM"

# Print Matcher Sheet URL
print("Go to {0} to view Estimator Sheet".format(estimator_sheet_url))

In [ ]:
# Allows window height to be larger
# Doesn't seem to work in Binder?
HTML('''
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:50000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>''');

In [ ]:
# Toggle Code Button
HTML('''<script>
code_show=true;
function code_toggle() {
    if (code_show){
    $('div.input').hide();
    } else {
    $('div.input').show();
    }
    code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the code."></form>
''')

# Update Journey Data

In [ ]:
# Creates current journeys update button and corresponding output
update_current_journeys_button = widgets.Button(description = "Update Current Journeys", layout = Layout(width = "200px"))
update_current_journeys_view_expression_button = widgets.Button(description = "Update Actual View and Expression Values", layout = Layout(width = "300px"))
out_update_current_journeys = widgets.Output()

def update_current_journeys(_) :
    with out_update_current_journeys :
        # Clears printed output
        clear_output()
        
        # Updates current journeys
        print("Updating Current Journeys", end = "\r")
        
        # Tells you how long it took to run
        # Initializes start time
        start_time = time.time()
        
        # Opens and clears current journeys sheet
        current_journeys_sheet = gs.open_by_url(estimator_sheet_url).worksheet("Current_Journeys")
        current_journeys_sheet.clear()
        
        # Updates Journey worksheet with cyfe board
        current_journeys_sheet.update("A1", gs.open_by_url(estimator_sheet_url).worksheet("URLs").get("B1")[0][0][1:], raw = False)
        
        # Waits for cell A1 to be the value "journey_id"
        while (current_journeys_sheet.acell("A1").value != "#N/A" and current_journeys_sheet.acell("A1").value != "journey_id") :
            time.sleep(5)
            
        # Copies cells to save journey
        current_journeys_sheet.update("A1:R", current_journeys_sheet.get("A1:R"), raw = False)
        
        # Clears previous results in Information
        gs.open_by_url(estimator_sheet_url).values_clear("Information!M4:R")
        gs.open_by_url(estimator_sheet_url).values_clear("Information!P2")
        
        # Prints out how long it took to run
        print("Updated Current Journeys Worksheet in {0:.3f} secs to run".format(time.time() - start_time))
        # Prints if cyfe board was empty
        if current_journeys_sheet.acell("A1").value == "#N/A" :
            print("No journeys loaded")
        
def update_view_expression(_) :
    with out_update_current_journeys :
        # Clears printed output
        clear_output()
        
        # Updates views and expressions
        print("Updating Actual View and Expression Values", end = "\r")
        
        # Tells you how long it took to run
        # Initializes start time
        start_time = time.time()
        
        # Opens current journeys sheet
        current_journeys_sheet = gs.open_by_url(estimator_sheet_url).worksheet("Current_Journeys")
        
        # Makes sure current journeys sheet has values already
        if (current_journeys_sheet.acell("A1").value != "journey_id") :
            print("Update Current Journeys First                                   ")
            return
        
        # Gets views and expressions from cyfe board
        views_expressions = pd.read_csv(io.StringIO(requests.get(gs.open_by_url(estimator_sheet_url).worksheet("URLs").get("B2")[0][0][1:]).content.decode('utf-8')))
        # Prints if cyfe board was empty
        if len(views_expressions) == 0 :
            print("No journeys loaded")
            return
        
        # Gets old current journeys data and removes views, expressions, and journey_id_2
        old_current_journeys = pd.DataFrame(current_journeys_sheet.get_all_records()).drop(["views", "expressions", "journey_id_2"], axis = 1)
        
        # Merges old current journeys with new views and expressions
        old_current_journeys = old_current_journeys.merge(views_expressions, how = "left", left_on = "journey_id", right_on = "journey_id_2").fillna("-")
        old_current_journeys["journey_id_2"] = old_current_journeys["journey_id"].values
        
        # Clears current journeys sheet and updates it
        current_journeys_sheet.clear()
        current_journeys_sheet.update([old_current_journeys.columns.values.tolist()] + old_current_journeys.values.tolist(), raw = False)
        
        # Clears old output views and expressions and displays the new ones
        gs.open_by_url(estimator_sheet_url).values_clear("Information!Q4:R")
        gs.open_by_url(estimator_sheet_url).worksheet("Information").update("Q4:R", old_current_journeys[["views", "expressions"]].values.tolist(), raw = False)
        
        # Prints out how long it took to run
        print("Updated Current Journeys Worksheet's Actual View and Expression Values in {0:.3f} secs to run".format(time.time() - start_time))
            
# Sets current journeys update buttons with functions
update_current_journeys_button.on_click(update_current_journeys)
update_current_journeys_view_expression_button.on_click(update_view_expression)

# Displays widgets
widgets.VBox([widgets.HBox([update_current_journeys_button,update_current_journeys_view_expression_button]), out_update_current_journeys])

# Estimate Number of Views and Expressions
#### Searches for similar journeys based on past data for the current journeys.
Similar journeys are journeys that have the exact same week day, exact same starting location, exact same number of men, and exact same number of jobs.<br>
Similar journeys also have a similar start time, similar end time, similar total time, similar total distance, similar start to end distance, similar volume, and similar price.<br>
These last 7 variables are not exact matches, but instead within +/- steps of the current journey's values.<br>
(i.e. For percent values, if a journey has a volume of 1.54 m^3, then initially it searches for journeys with volumes between 1.5015 m^3 and 1.5785 m^3 which corresponds to +/- 2.5% of the journey's volume)<br>
(i.e. For fixed values, if a journey has a volume of 1.54 m^3, then initially it searches for journeys with volumes between 1.04 m^3 and 2.04 m^3 which corresponds to +/- 0.5 m^3)<br>
If not enough similar journeys are found in this initial step range, then the range is increased until a sufficient number of similar journeys are found.<br>
<br>
For fixed values, these range increments are used: 15 minutes for start, end, and total time; 10 km for total distance; 1 km for start to end distance; 0.1 m^3 for volume; and £1 for TP amount

In [ ]:
# Creates load current and past journeys button and corresponding output
load_current_past_journeys_button = widgets.Button(description = "Load Current and Past Journeys", layout = Layout(width = "250px"))
out_load_current_past_journeys = widgets.Output()

# Initializes current_journeys and past_journeys
current_journeys = None
past_journeys = None

def load_current_past_journeys(_) :
    # Allows current_journeys and past_journeys to be saved outside the function
    global current_journeys, past_journeys
    
    with out_load_current_past_journeys :
        # Clears printed output
        clear_output()
        
        # Gets current journey data
        print("Loading Current Journeys", end = "\r")
        current_journeys = pd.DataFrame(gs.open_by_url(estimator_sheet_url).worksheet("Current_Journeys").get_all_records()).fillna(0.0)
        
        # Handles 0 journeys case
        if len(current_journeys.index) == 0 :
            current_journeys = None
            print("No Current Journeys Found")
        else :
            print("Current Journeys Loaded: {0} Journeys".format(len(current_journeys.index)))
        
        # Gets past journey data
        print("Loading Past Journeys", end = "\r")
        past_journeys = pd.DataFrame(gs.open_by_url(estimator_sheet_url).worksheet("Past_Journeys").get_all_records(head = 3)).fillna(0.0)
        
        # Handles 0 journeys case
        if len(past_journeys.index) == 0 :
            past_journeys = None
            print("No Past Journeys Found")
        else :
            # Adds a month to past_journeys
            if "month" not in past_journeys.columns :
                past_journeys["month"] = [past_journeys.strip().split("/" if "/" in date else "-")[1] for date in past_journeys["date"]]
            # Removes all journeys that were returns
            if "return" in past_journeys.columns :
                past_journeys = past_journeys.where(past_journeys["return"] != 1).dropna()
            print("Past Journeys Loaded: {0} Journeys ranging from {1} to {2}".format(len(past_journeys.index), min(past_journeys["date"]), max(past_journeys["date"])))
            
            if (date.today() - date(*[int(x) for x in max(past_journeys["date"]).split("-")])).days >= 14 :
                print("\n\033[1mConsider updating past journeys on Estimator Sheet\033[0m")

# Sets load current and past journeys button with function
load_current_past_journeys_button.on_click(load_current_past_journeys)

# Displays widgets
widgets.VBox([load_current_past_journeys_button, out_load_current_past_journeys])

In [ ]:
# Creates journey info dropdown box
# Does this if current_journeys is loaded or displays "Try to re-run cell" if current_journeys is not loaded
if current_journeys is not None :
    journey_info = widgets.Dropdown(
        options = np.insert(np.array(current_journeys["journey_id"].sort_values().unique(), dtype = object), 0, "Select journey"),
        value = "Select journey",
        description = "Journey:",
        style = {"description_width":"initial"})
else :
    journey_info = widgets.Dropdown(
        options = ["Select journey", "Try to re-run cell"],
        value = "Select journey",
        description = "Journey:",
        style = {"description_width":"initial"})
    
# Creates journey info button and corresponding output
journey_info_button = widgets.Button(description = "Display Journey Info", layout = Layout(width = "160px"))
out_journey_info = widgets.Output()

def format_time(time) :
    # Formats time in HH:MM for all times
    try :
        format_time = float(time)
        format_hour = int(format_time * 24)
        format_min = int((format_time * 24 - format_hour) * 60)

        return "{0:02d}:{1:02d}".format(format_hour, format_min)
    except :
        return time

def string_handling(x, float_int) :
    # Handles string if "-"
    if x == "-" :
        return x
    
    # Handles string if there are commas and converts string to float or int
    try :
        result = float_int(x)
    except :
        result = float_int(x.replace(",",""))
        
    return result

def display_journey_info(_) :
    with out_journey_info :
        # Clears printed output
        clear_output()
        
        # Makes sure an actual journey is selected
        if journey_info.value != "Select journey" and journey_info.value != "Try to re-run cell" and journey_info.value != None :
            # Gets the journey data
            journey = current_journeys.where(current_journeys["journey_id"] == journey_info.value).dropna(how = "all").iloc[0]
            
            # Prints journey data information
            print(u"Journey ID: {0}\nDate: {1}\nTP Amount: \xa3{2:.2f}\nNumber of Men: {3}\nNumber of Jobs: {4}\nStart to End Distance: {5:.2f} km\nTotal Distance: {6:.2f} km".format(string_handling(journey["journey_id"], int),
                                                                                                                                                                                       journey["date"],
                                                                                                                                                                                       string_handling(journey["price"], float),
                                                                                                                                                                                       string_handling(journey["number_of_men"], int),
                                                                                                                                                                                       string_handling(journey["number_of_jobs"], int),
                                                                                                                                                                                       string_handling(journey["finish_distance"], float), 
                                                                                                                                                                                       string_handling(journey["km"], float)))
            print("Start Location: {0}\nStart Time: {1}\nEnd Time: {2}\nTotal Time: {3:.2f} hours\nMax Volume: {4:.2f} m^3\nMax Weight: {5} kg".format(journey["start_location"],
                                                                                                                                                       format_time(journey["start_time"]), 
                                                                                                                                                       format_time(journey["end_time"]),
                                                                                                                                                       string_handling(journey["total_time_day"], float) * 24,
                                                                                                                                                       string_handling(journey["volume"], float), 
                                                                                                                                                       string_handling(journey["kg"], float)))
            # Prints out estimates if calculated
            if estimates is not None :
                # Gets the estimates
                journey_estimate = estimates.where(estimates["journey_id"] == journey_info.value).dropna(how = "all").iloc[0]
                # Reformat data_overlap
                journey_estimate["data_overlap"]["job_id"] = [int(journey_id) for journey_id in journey_estimate["data_overlap"]["job_id"]]
                journey_estimate["data_overlap"]["start_time"] = [format_time(time) for time in journey_estimate["data_overlap"]["start_time"]]
                journey_estimate["data_overlap"]["end_time"] = [format_time(time) for time in journey_estimate["data_overlap"]["end_time"]]
                journey_estimate["data_overlap"]["total_time_day"] = [time * 24 for time in journey_estimate["data_overlap"]["total_time_day"]]
                
                # Prints out estimates and actual
                if not last_run :
                    print("\n{0} Similar Journeys within {1:.1f}% of Values\n".format(journey_estimate["number_similar"], journey_estimate["step_size"] * 100))
                else :
                    print("\n{0} Similar Journeys within {1} Steps of Values\n".format(journey_estimate["number_similar"], journey_estimate["step_size"]))
                print("Estimated Views: {0}\nActual Views: {1}".format("{0:.1f}".format(journey_estimate["pred_views"]) if journey_estimate["pred_views"] != "-" else journey_estimate["pred_views"], journey["views"]))
                print("Estimated Expressions: {0}\nActual Expressions: {1}\n".format("{0:.1f}".format(journey_estimate["pred_expressions"]) if journey_estimate["pred_expressions"] != "-" else journey_estimate["pred_expressions"], journey["expressions"]))
                
                # Prints out data_overlap
                print("Similar Journeys:")
                if len(journey_estimate["data_overlap"].index) > 0 :
                    display(journey_estimate["data_overlap"][["job_id", "date", "views", "expressions", "journey_tp_price", "number_of_men", 
                                                              "active_listing_count", "start_end", "km", "start_n1", "start_time", "end_time",
                                                              "total_time_day", "m3", "kg"]].rename(
                                                columns = {"job_id" : "Journey ID", "date" : "Date", "views" : "Views", "expressions" : "Expressions", "journey_tp_price" : u"TP Amount (\xa3)", "number_of_men" : "Number of Men", 
                                                           "active_listing_count" : "Number of Jobs", "start_end" : "Start to End Distance (km)", "km" : "Total Distance (km)", "start_n1" : "Start Location", "start_time" : "Start Time", "end_time" : "End Time",
                                                           "total_time_day" : "Total Time (hour)", "m3" : "Max Volume (m^3)", "kg" : "Max Weight (kg)"}))
                else :
                    print("None")
            
# Sets the journey info button with function
journey_info_button.on_click(display_journey_info)

# Displays widgets
widgets.VBox([widgets.HBox([journey_info, journey_info_button]), out_journey_info])

In [ ]:
# Creates number of random sorting iterations
min_number_similar = widgets.IntText(
                     value = 5,
                     description = "Minimum number of similar journeys:",
                     style = {"description_width":"initial"})

# Creates fixed_percent checkbox
fixed_percent = widgets.Checkbox(
                value = False, 
                description = "Fixed Values (checked) or Percent Values (unchecked)?",
                disabled = False,
                layout = Layout(width = "500px"))

# Initializes estimates
estimates = None
estimates_columns = ["journey_id", "pred_views", "pred_expressions", "number_similar", "step_size", "data_overlap"]

# Initializes step_increment and step_max
step_increment = None
step_max = None

# Step increments for each variable
steps = {
    "start_time" : 0.01041666666666666666666666666667, # 15 min
    "end_time" : 0.01041666666666666666666666666667, # 15 min
    "total_time_day" : 0.01041666666666666666666666666667, # 15 min
    "km" : 10,
    "finish_distance" : 1,
    "volume" : 0.1,
    "price" : 1
}

# Converts journey column to data column
key_convert = {
    "start_time" : "start_time",
    "end_time" : "end_time",
    "total_time_day" : "total_time_day",
    "km" : "km",
    "finish_distance" : "start_end",
    "volume" : "m3",
    "price" : "journey_tp_price"
}

# Sets last run scenario
last_run = None

def estimate_number(journey, step, iteration, number_iterations) :
    # Allows estimates to be saved outside of the function
    global estimates, estimates_columns
    
    # Gets the overlapping past data
    data_overlap = data_overlap_prime(journey, step)
    
    if not fixed_percent.value :
        print("Working on journey {0}: {1} of {2} journeys (Step: {3:.1f}%, Number Similar: {4})        ".format(string_handling(journey["journey_id"], int), 
                                                                                                                 iteration + 1, 
                                                                                                                 number_iterations,
                                                                                                                 step * 100, 
                                                                                                                 len(data_overlap.index)), end = "\r")
    else :
        print("Working on journey {0}: {1} of {2} journeys (Step: {3}, Number Similar: {4})        ".format(string_handling(journey["journey_id"], int), 
                                                                                                            iteration + 1, 
                                                                                                            number_iterations,
                                                                                                            step, 
                                                                                                            len(data_overlap.index)), end = "\r")
    
    if len(data_overlap.index) < min_number_similar.value and step < step_max :
        # Estimates views and expressions with larger step size
        estimate_number(journey, step + step_increment, iteration, number_iterations)
    else :  
        # Calculates the number of views and expressions
        count_view = data_overlap["views"].median() if not np.isnan(data_overlap["views"].mean()) else "-"
        count_express = data_overlap["expressions"].median() if not np.isnan(data_overlap["expressions"].mean()) else "-"
        
        # Saves the number of views and expressions alongsize the step size, number of similar, and overlapping data
        estimates = estimates.append(pd.DataFrame([[journey["journey_id"], count_view, count_express, len(data_overlap.index), step, data_overlap]], columns = estimates_columns))
        
def data_overlap_prime(journey, step) :
    # All variables with exact matches
    data = past_journeys.where(past_journeys["week_day"] == string_handling(journey["week_day"], int)).dropna(how = "all")
    data = data.where(data["start_n1"] == journey["start_location"]).dropna(how = "all")
    data = data.where(data["number_of_men"] == string_handling(journey["number_of_men"], int)).dropna(how = "all")
    data = data.where(data["active_listing_count"] == string_handling(journey["number_of_jobs"], int)).dropna(how = "all")
    
    # All variables with step size looking
    for key in steps.keys() :
        if fixed_percent.value :
            data = data.where(np.logical_and(data[key_convert[key]] >= string_handling(journey[key], float) - step * steps[key], 
                                             data[key_convert[key]] <= string_handling(journey[key], float) + step * steps[key])).dropna(how = "all")
        else :
            data = data.where(np.logical_and(data[key_convert[key]] >= string_handling(journey[key], float) * (1 - step), 
                                             data[key_convert[key]] <= string_handling(journey[key], float) * (1 + step))).dropna(how = "all")
        
    return data

# Displays widgets
widgets.VBox([min_number_similar, fixed_percent])

In [ ]:
# Create estimate view and expression button with corresponding button
estimate_views_expressions_button = widgets.Button(description = "Estimate View and Expression Values", layout = Layout(width = "300px"))
out_estimate_views_expressions = widgets.Output()

def estimate_views_expressions(_) :
    # Allows estimates to be saved outside of the function
    global estimates, estimates_columns, last_run, step_increment, step_max
    
    with out_estimate_views_expressions :
        # Clears printed output
        clear_output()
        
        # Makes sure min_number_similar is appropriate
        if min_number_similar.value <= 0 :
            print("Select a valid number of similar journeys")
            return
        
        # Makes sure current_journeys and past_journeys loaded
        if current_journeys is None or past_journeys is None :
            print("Load Current and Past Journeys First")
            return
        
        # Initializes estimates
        estimates = pd.DataFrame(columns = estimates_columns)
        
        # Tells you how long it took to run
        # Initializes start time
        start_time = time.time()
        
        # Sets last run scenario
        last_run = fixed_percent.value
        
        # Amount to increment step by and max step
        step_increment = 1 if fixed_percent.value else 0.025
        step_max = 1000 if fixed_percent.value else 5
        
        for i,journey in current_journeys.iterrows() :
            estimate_number(journey, step_increment, i, len(current_journeys.index))
            if not fixed_percent.value :
                print("Completed journey {0}: {1} of {2} journeys (Step: {3:.1f}%, Number Similar: {4})        ".format(string_handling(journey["journey_id"], int),
                                                                                                                        i + 1, 
                                                                                                                        len(current_journeys.index),
                                                                                                                        estimates["step_size"].iloc[i] * 100,
                                                                                                                        estimates["number_similar"].iloc[i]))
            else :
                print("Completed journey {0}: {1} of {2} journeys (Step: {3}, Number Similar: {4})        ".format(string_handling(journey["journey_id"], int),
                                                                                                                   i + 1, 
                                                                                                                   len(current_journeys.index),
                                                                                                                   estimates["step_size"].iloc[i],
                                                                                                                   estimates["number_similar"].iloc[i]))
            
        # Clears the printed output and prints that estimates is done
        clear_output()
        print("\nCompleted {0} journeys in {1:.3f} sec".format(len(current_journeys.index), time.time() - start_time))
        
# Sets the estimate view and expression button with function
estimate_views_expressions_button.on_click(estimate_views_expressions)

# Display widgets
widgets.VBox([estimate_views_expressions_button, out_estimate_views_expressions])

# Save Number of Views and Expressions

In [ ]:
# Creates estimator display and estimator save buttons with corresponding outputs
estimator_display_button = widgets.Button(description = "Display Results", layout = Layout(width = "150px"))
estimator_save_button = widgets.Button(description = "Save Results", layout = Layout(width = "150px"))
out_estimator_display = widgets.Output()

def estimator_display(_) :
    with out_estimator_display :
        # Clears printed output
        clear_output()
        
        # Makes sure estimator is run first
        if estimates is None :
            print("Estimate Views and Expressions First")
            return
            
        # Prints estimator data header
        print("{0:11} {1:16} {2:13} {3:22} {4:19} {5:15} {6:10}".format("Journey ID", "Predicted Views", "Actual Views", "Predicted Expressions", "Actual Expressions", "Number Similar", "Step Size"))
        for _,estimate in estimates.sort_values(by = "journey_id").iterrows() :
            # Gets corresponding journey data
            journey = current_journeys.where(current_journeys["journey_id"] == estimate["journey_id"]).dropna(how = "all").iloc[0]
            
            # Prints estimate data
            if not last_run :
                print("{0:11} {1:16} {2:13} {3:22} {4:19} {5:15} {6:10}".format("{}".format(string_handling(estimate["journey_id"], int)), 
                                                                                "{0:.1f}".format(estimate["pred_views"]) if estimate["pred_views"] != "-" else estimate["pred_views"],
                                                                                "{}".format(string_handling(journey["views"], int)),
                                                                                "{0:.1f}".format(estimate["pred_expressions"]) if estimate["pred_expressions"] != "-" else estimate["pred_expressions"],
                                                                                "{}".format(string_handling(journey["expressions"], int)),
                                                                                "{}".format(string_handling(estimate["number_similar"], int)),
                                                                                "{0:.1f}%".format(estimate["step_size"] * 100)))
            else :
                print("{0:11} {1:16} {2:13} {3:22} {4:19} {5:15} {6:10}".format("{}".format(string_handling(estimate["journey_id"], int)), 
                                                                                "{0:.1f}".format(estimate["pred_views"]) if estimate["pred_views"] != "-" else estimate["pred_views"],
                                                                                "{}".format(string_handling(journey["views"], int)),
                                                                                "{0:.1f}".format(estimate["pred_expressions"]) if estimate["pred_expressions"] != "-" else estimate["pred_expressions"],
                                                                                "{}".format(string_handling(journey["expressions"], int)),
                                                                                "{}".format(string_handling(estimate["number_similar"], int)),
                                                                                "{}".format(estimate["step_size"])))
            
def estimator_save(_) :
    with out_estimator_display :
        # Clears output
        clear_output()
        
        # Makes sure estimator is run first
        if estimates is None :
            print("Estimate Views and Expressions First")
            return
        
        print("Saving Estimator Results", end = "\r")
        
        # Opens Estimator Sheet's Information subsection
        info_sheet = gs.open_by_url(estimator_sheet_url).worksheet("Information")
        # Clears previous results in Information
        gs.open_by_url(estimator_sheet_url).values_clear("Information!M4:R")
        gs.open_by_url(estimator_sheet_url).values_clear("Information!P2")
        
        # Initializes output from estimator
        estimates_output = estimates.drop(["journey_id", "data_overlap"], axis = 1)
        estimates_output["actual_views"] = current_journeys["views"].values
        estimates_output["actual_expressions"] = current_journeys["expressions"].values
        
        # Saves output from estimator to Estimator Sheet's Information subsection
        info_sheet.update("M4:R", estimates_output.values.tolist(), raw = False)
        info_sheet.update("P2", "Fixed Values" if last_run else "Percent")
        
        print("Saved Estimator Results              ")
        # Prints Matcher Sheet url
        print("Go to: {0} to view".format(estimator_sheet_url))
        
# Sets estimator display and estimator save buttons with their corresponding functions
estimator_display_button.on_click(estimator_display)
estimator_save_button.on_click(estimator_save)

# Displays widgets
widgets.VBox([widgets.HBox([estimator_display_button, estimator_save_button]), out_estimator_display])

# Done